In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

#make new folder in drive
new_folder_path = '/content/drive/My Drive/graduation2024'
os.makedirs(new_folder_path, exist_ok=True)

In [ ]:
cd "/content/drive/My Drive/graduation2024"

/content/drive/My Drive/graduation2024


In [ ]:
!git clone https://github.com/SKplanet/Dialog-KoELECTRA.git

In [ ]:
!pip install transformers

tokeninzer 수정 위해 기존 KoElectra Vocab에 단어 추가 - '파이썬'만 추가


In [ ]:
pip install autocorrect # 오타 교정

In [ ]:
pip install googletrans==4.0.0-rc1 # 언어 통일(papagoAPI 대용)

In [ ]:
from autocorrect import Speller # 오타 교정
# maek => make
import requests
import googletrans # 번역

def fix_typo(sentence, language): # 오타 교정
  spell = Speller(lang=language)

  # split the sentence into words
  words = sentence.split()

  # find and fix typo
  correct_words = [spell(word)+' ' for word in words]

  # make a sentence
  correct_sentence = ''.join(correct_words)
  return correct_sentence


# 타 언어 제거, 번역
# using 파이썬 modify this code => using python modify this code

def mono_lang(sentence, language): # input sentence, 'en', 'ko' ...etc
  # general translation
  source_lang = 'ko'
  target_lang = language
  text = sentence

  # papago API
  client_id = 'EMmKltfYKVWFG61jU5OW'
  client_secret = 'i5npjLvENG'

  # API 엔드포인트 및 헤더 설정
  url = 'https://openapi.naver.com/v1/papago/n2mt'
  headers = {
      'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
      'X-Naver-Client-Id': client_id,
      'X-Naver-Client-Secret': client_secret,
  }

  # en, ko, 번역 텍스트
  data = {
      'source': source_lang,
      'target': target_lang,
      'text': text,
  }

  # API 요청
  response = requests.post(url, headers=headers, data=data)

  # JSON 응답 파싱
  result = response.json()

  # 번역된 텍스트 반환
  translated_text = result['message']['result']['translatedText']
  print('1차 번역 :',translated_text) # 한->영

  data = {
      'source': target_lang,
      'target': source_lang,
      'text': translated_text,
  }

  # API 요청2
  response = requests.post(url, headers=headers, data=data)

  # JSON 응답 파싱2
  result = response.json()

  translated_text = result['message']['result']['translatedText']
  print('2차 번역 :',translated_text) # 영->한

  return translated_text


In [ ]:
import torch

def detokenize(tokens): # 띄어쓰기 제공
    words = []
    current_word = ""
    for token in tokens:
        if token.startswith("##"):
            current_word += token[2:]
        else:
            if current_word:
                words.append(current_word)
            current_word = token

    if current_word:
        words.append(current_word)

    return " ".join(words)


def sentence_split_generate(input_text):
  result = tokenizer.tokenize(input_text)
  print('result : ', result)
  # ['', '[CLS]', '1', '##에', '##서', '10', '##까', '##지', '더', '##하', '##고', ',', '그', '값', '##을', '10', '##으로', '나누', '##는', '파이', '##썬', '코드', '##를', '만들', '##어', '##줘', '.']


# 문장의 목적을 따로 분류하도록 모델을 fine-tuning시켜 강조?


  return_sentence = []
  current_sent = ""

  remove_point = ['[CLS]', '[SEP]']

  for word in result:

    if word in remove_point: # 앞뒤 CLS, SEP 삭제
      continue

  final_result = detokenize(result) # 띄어쓰기 포함해서 문장 합치기

  # if word.endswith('.') or word.endswith(','): # .나 ,로 문장에 줄바꿈
  #   return_sentence.append(current_sent.strip())
  #   current_sent = ""
  # print('return : ', return_sentence)
  return final_result

def generate_custom_sentence(prompt, max_length=50, temperature=1.0): # dialog-koelectra이용해 문장 이쁘게 다듬기(최종)
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output_ids = model.generate(input_ids, max_length=max_length, temperature=temperature, num_beams=5)
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return generated_text


In [ ]:
from transformers import ElectraTokenizer, ElectraForSequenceClassification
import torch


# chatGPT 프롬프트가 대화형이므로 한국어 대화형에 최적화된 모델인 skplanet - dialog-koElectra를 사용
tokenizer = ElectraTokenizer.from_pretrained("skplanet/dialog-koelectra-small-discriminator")
model = ElectraForSequenceClassification.from_pretrained("skplanet/dialog-koelectra-small-discriminator")


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at skplanet/dialog-koelectra-small-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
## main ##

def set_seed(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# 랜덤 시드
set_seed(42)

#update tokenizer process
add_token_num = tokenizer.add_tokens(['파이썬'])# add token in vocab

# update tokenizer
model.resize_token_embeddings(len(tokenizer))

# Sentence to be paraphrased => 여기를 input값으로 수정하면 됨!!
input_sentence = "chatGPT가 짜준 코드를 웹크롤링해서 불러온 다음에, 컴파일러에 실행시키고 오류를 재귀적으로 고쳐서 완벽한 코드를 만드는 파이썬 코드를 알려줘 " # => make a python code which adds from 1 to 10 (이정도까지 오타가 나진 않을거같긴함)

# Tokenize the sentence / preprocessing
language = 'en' # target language
fixed_typo_sentence = fix_typo(input_sentence, language) # 특수문자, 오타, 타 언어, 띄어쓰기 교정
print("오타 교정 : ",fixed_typo_sentence)
fixed_language = mono_lang(fixed_typo_sentence, language) # 타 언어 제거 - (input언어가 한국어인 경우만 상정) 파파고 api 이용->번역 2회
print('언어 통일 : ', fixed_language)

# conjunctions = ['그리고', '또한']  # 문장 분할시 추가 접속사들

fixed_lang = sentence_split_generate(fixed_language) # 문장 분할해서 새 문장 생성(줄바꿈)
print("문장 분할 : ",fixed_lang)


# 추가 문맥제공, 강조, trigger sentence 추가 / 문맥, 강조 후에 추가
trigger_sent = "\n이 과정을 순서대로 알려줘" # 변형 가능 // process in order / print in order . . .

# gui를 사용해 맥락 제공 -> 맥락에 맞는 버튼을 눌러서 트리거 문장 추가
triggerSentences = ['이 과정을 순서대로 알려줘.',
                    '맞춤법이 틀린 부분을 수정해줘.',
                    '모든 코드에 주석을 달아줘.'] # 추가될 수 있음

final_output = fixed_lang + trigger_sent


# 문장 분할된 값을 모델에 넣어서 문장 이쁘게 다듬기
print("\nfinal output using dialog-koElectra : ", final_output)


오타 교정 :  chatGPT가 짜준 코드를 웹크롤링해서 불러온 다음에, 컴파일러에 실행시키고 오류를 재귀적으로 고쳐서 완벽한 코드를 만드는 파이썬 코드를 알려줘 
1차 번역 : After webcrolling the code that chatGPT put together, let me know the Python code that makes perfect code by executing it in the compiler and recursively correcting errors
2차 번역 : chatGPT들이 정리한 코드를 웹 크롤링한 후 컴파일러에서 실행하여 오류를 재귀적으로 수정하여 완벽한 코드를 만드는 파이썬 코드를 알려주세요
언어 통일 :  chatGPT들이 정리한 코드를 웹 크롤링한 후 컴파일러에서 실행하여 오류를 재귀적으로 수정하여 완벽한 코드를 만드는 파이썬 코드를 알려주세요
result :  ['c', '##ha', '##t', '##G', '##P', '##T', '##들이', '정리', '##한', '코드', '##를', '웹', '크', '##롤', '##링', '##한', '후', '컴', '##파일', '##러', '##에서', '실행', '##하', '##여', '오류', '##를', '재', '##귀', '##적', '##으로', '수정', '##하', '##여', '완벽', '##한', '코드', '##를', '만드', '##는', '파이썬', '코드', '##를', '알려', '##주세요']
문장 분할 :  chatGPT들이 정리한 코드를 웹 크롤링한 후 컴파일러에서 실행하여 오류를 재귀적으로 수정하여 완벽한 코드를 만드는 파이썬 코드를 알려주세요

final output using dialog-koElectra :  chatGPT들이 정리한 코드를 웹 크롤링한 후 컴파일러에서 실행하여 오류를 재귀적으로 수정하여 완벽한 코드를 만드는 파이썬 코드를 알려주세요
이 과정을 순서대로 알려줘


@misc{DialogKoELECTRA,
  author       = {Wonchul Kim and Junseok Kim and Okkyun Jeong},
  title        = {Dialog-KoELECTRA: Korean conversational language model based on ELECTRA model},
  howpublished = {\url{https://github.com/skplanet/Dialog-KoELECTRA}},
  year         = {2021},
}

openai API



In [ ]:
import openai

# OpenAI API 키 설정
openai.api_key = 'YOUR_API_KEY'

def prompt_engineering(prompt, max_tokens=50, temperature=0.7):
    # GPT 모델에 입력할 prompt 설정
    input_prompt = f"프롬프트 엔지니어링: {prompt}"

    # OpenAI API 호출
    response = openai.Completion.create(
        engine="text-davinci-002",  # 적절한 엔진 선택
        prompt=input_prompt,
        max_tokens=max_tokens,
        temperature=temperature
    )

    # 생성된 텍스트 반환
    return response['choices'][0]['text'].strip()

# 프롬프트 엔지니어링을 테스트할 문장
input_sentence = "인공지능에 대해 설명해봐."

# 프롬프트 엔지니어링 함수 호출
output_sentence = prompt_engineering(input_sentence)

# 결과 출력
print(f"입력 문장: {input_sentence}")
print(f"프롬프트 엔지니어링 결과: {output_sentence}")



<!-- # @misc{DialogKoELECTRA,
#   author       = {Wonchul Kim and Junseok Kim and Okkyun Jeong},
#   title        = {Dialog-KoELECTRA: Korean conversational language model based on ELECTRA model},
#   howpublished = {\url{https://github.com/skplanet/Dialog-KoELECTRA}},
#   year         = {2021},
# } -->